In [25]:
import cv2
import mediapipe as mp
import numpy as np

In [28]:
# 안경 이미지 로드
mask_img = cv2.imread('4892.png_300.png', cv2.IMREAD_UNCHANGED)  # 알파 채널을 포함하여 로드
print(mask_img.shape)

_, _, _, alpha_channel = cv2.split(mask_img)

h, w = mask_img.shape[:2] # shape 에서 반환되는 값은 h, w 순서
resized_mask = cv2.resize(alpha_channel, (w, h), interpolation=cv2.INTER_AREA)
masked_img_1 = cv2.bitwise_and(mask_img, mask_img, mask=resized_mask)

cv2.imshow("mask_img", masked_img_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

(300, 300, 3)


ValueError: not enough values to unpack (expected 4, got 3)

In [14]:
# MediaPipe 얼굴 검출 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

mask_img = cv2.imread('mask.png', cv2.IMREAD_UNCHANGED)  # 알파 채널을 포함하여 로드

# 웹캠 캡처 객체 생성
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break  # 프레임을 제대로 읽지 못하면 루프 탈출

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 얼굴 랜드마크 검출
    results = face_mesh.process(rgb_frame)

    # 얼굴 랜드마크에 따라 안경 오버레이
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 눈의 위치를 기준으로 안경 위치 조정 및 크기 조정 로직 필요 (여기서는 간단히 처리)
            # 예시로 랜드마크를 사용하지 않고 고정된 위치에 적용하는 방식으로 단순화
            # 실제 사용시에는 랜드마크를 기반으로 안경의 위치와 크기를 동적으로 조정해야 합니다.
            
            # 안경 이미지의 알파 채널과 RGB 채널 분리
            mask_alpha = mask_img[:, :, 3] / 255.0
            mask_color = mask_img[:, :, :3]

            # 안경을 적용할 위치 계산 (여기서는 전체 이미지 크기를 기준으로 간단히 설정)
            x, y, w, h = 100, 100, 200, 100  # 예시 위치 및 크기
            mask_resized = cv2.resize(mask_color, (w, h))
            alpha_resized = cv2.resize(mask_alpha, (w, h), interpolation=cv2.INTER_AREA)

            # 알파 블렌딩을 통한 안경 이미지 합성
            for c in range(0, 2):
                frame[y:y+h, x:x+w, c] = alpha_resized * mask_resized[:, :, c] + (1-alpha_resized) * frame[y:y+h, x:x+w, c]

    # 결과 표시
    cv2.imshow('Glasses Overlay', frame)

    # 'q'를 누르면 루프 탈출
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


IndexError: index 3 is out of bounds for axis 2 with size 3